In [1]:
# from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
# import os
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision import models,transforms,datasets
import torch
import bcolz
import time
%matplotlib inline

In [2]:
import imp
import utils; imp.reload(utils)
from utils import *

### Load preconvoluted features

In [3]:
data_dir = '/home/foodlovers/FoodVisor/data/images'

In [4]:
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x))
         for x in ['train', 'test']}

In [6]:
dsets['train'].imgs[:5]

[('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_0.jpg', 0),
 ('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_100.jpg',
  0),
 ('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_101.jpg',
  0),
 ('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_102.jpg',
  0),
 ('/home/foodlovers/FoodVisor/data/images/train/apple_pie/apple_pie_104.jpg',
  0)]

In [5]:
conv_feat_train = load_array('/home/foodlovers/vgg16/conv_feat_train.bc')
labels_train = load_array('/home/foodlovers/vgg16/labels_train.bc')
conv_feat_val = load_array('/home/foodlovers/vgg16/conv_feat_val.bc')
labels_val = load_array('/home/foodlovers/vgg16/labels_val.bc')

In [32]:
conv_feat_train.shape

(67988, 512, 7, 7)

In [14]:
len(dsets['train'].imgs) == len(conv_feat_train)

True

In [8]:
print(labels_train[:10])
print([x[1] for x in dsets['train'].imgs[:10]])

[0 0 0 0 0 0 0 0 0 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
for i,label in enumerate(labels_train) :
    if label != dsets['train'].imgs[i][1] :
        print("Wrong !!!")
for i,label in enumerate(labels_val) :
    if label != dsets['test'].imgs[i][1] :
        print("Wrong !!!")
print("True !")

True !


### Load corresponding text files

In [6]:
# Preprocessed textes in Food_Text_Classifier.ipynb
text_dir = '/home/foodlovers/FoodVisor/data/processed_texts_txt'

In [24]:
file_path = dsets['train'].imgs[1][0]
print(file_path.split("/")[-2:])
print("/".join(file_path.split("/")[-2:])[:-3]+'txt')
path_text = text_dir+'/'+"/".join(file_path.split("/")[-2:])[:-3]+'txt'
print(path_text)
text_file = open(path_text, "r")
text = text_file.read()
text_file.close()
print(text)

['apple_pie', 'apple_pie_100.jpg']
apple_pie/apple_pie_100.txt
/home/foodlovers/FoodVisor/data/processed_texts_txt/apple_pie/apple_pie_100.txt
My account My newslett My groceri list My save custom care log My box submit log subscrib digit edit give gift cookbook subscrib best BY cours appet breakfast dessert dinner side cours BY beef chicken strawberri tomato watermelon BY style grill ahead potluck quick slow cooker style healthi budget video partner advanc see submit contest save best deal subscrib today holiday holiday juli th labor day celebr summer entertain birthday holiday celebr fourth OF juli flagshap grill classic american pie red white blue dessert juli th commun commun favorit forum group commun facebook pinterest tumblr google twitter popular pin check pinterest follow school school Us find show school meet staff faq school find show attend show fun uniqu night learn skill memori 'll last lifetim find show cookbook across america discov countri best flavor shop cookbook acr

In [7]:
text_train = []
text_val = []
# a lot of file don't contain any text
valid_conv_feat_train = []
valid_labels_train = []
valid_conv_feat_val = []
valid_labels_val = []

unvalid_count = 0
for i in range(len(conv_feat_train)) :
    try :
        file_path = dsets['train'].imgs[i][0]
        path_text = text_dir+'/'+"/".join(file_path.split("/")[-2:])[:-3]+'txt'
        text_file = open(path_text, "r")
        text = text_file.read()
        text_file.close()
        text_train.append(text)
        valid_conv_feat_train.append(conv_feat_train[i])
        valid_labels_train.append(labels_train[i])
    except FileNotFoundError :
        # print(file_path)
        unvalid_count += 1
    
for i in range(len(conv_feat_val)) :
    try :
        file_path = dsets['test'].imgs[i][0]
        path_text = text_dir+'/'+"/".join(file_path.split("/")[-2:])[:-3]+'txt'
        text_file = open(path_text, "r")
        text = text_file.read()
        text_file.close()
        text_val.append(text)
        valid_conv_feat_val.append(conv_feat_val[i])
        valid_labels_val.append(labels_val[i])
    except FileNotFoundError :
        # print(file_path)
        unvalid_count += 1
        
print(unvalid_count," images don't have text file")

3908  images don't have text file


In [9]:
print("train:",len(text_train),"test:",len(text_val))

train: 65101 test: 21695


### Transform text file into TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(binary=True)
text_feat_train = tfidf.fit_transform(text_train)
#.astype('float16')
text_feat_val = tfidf.transform(text_val)

In [11]:
text_feat_train.shape

(65101, 60033)

In [10]:
valid_conv_feat_train = np.asarray(valid_conv_feat_train).reshape(len(valid_conv_feat_train),-1)
valid_conv_feat_val = np.asarray(valid_conv_feat_val).reshape(len(valid_conv_feat_val),-1)

### Test image features with linearSVC

In [41]:
from sklearn.svm import LinearSVC

In [43]:
%%time
clf = LinearSVC()
clf.fit(valid_conv_feat_train, valid_labels_train)

CPU times: user 21min 36s, sys: 3.93 s, total: 21min 40s
Wall time: 21min 40s


In [45]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(valid_conv_feat_val)
accuracy = accuracy_score(valid_labels_val, y_pred)
print("Accuracy : ", accuracy)

Accuracy :  0.3752477529384651


## Build my model

Count les nombres de parametres ...

In [13]:
print("Text features shape:",text_feat_train.shape,"Image features shape:",valid_conv_feat_train.shape)

Text features shape: (65101, 60033) Image features shape: (65101, 25088)


In [11]:
use_gpu = torch.cuda.is_available()

### Train textual features with a sequential model

In [12]:
class myNet(nn.Module):
    def __init__(self):
        super(myNet, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(60033, 1024),
            nn.ReLU(True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(True),
            nn.Dropout(0.3),
            nn.Linear(512, 101),
        ) 

    def forward(self, x):
        x = self.classifier(x)
        return x

In [13]:
net1 = myNet()

if use_gpu:
    net1 = net1.cuda()

In [15]:
def data_gen_1(data,labels,batch_size=64,shuffle=True):
    labels = np.array(labels)
    if shuffle:
        index = np.random.permutation(len(labels))
        data = data[index]
        labels = labels[index]
    for idx in range(0,len(labels),batch_size):
        if type(data).__module__ == 'numpy' :
            yield(data[idx:idx+batch_size],labels[idx:idx+batch_size],
             int(len(labels) / batch_size) + (len(labels) % batch_size > 0))
        else :
            yield(data[idx:idx+batch_size].toarray(),labels[idx:idx+batch_size],
             int(len(labels) / batch_size) + (len(labels) % batch_size > 0))

In [16]:
def train_model_1(model, criterion, batch_size = 64,
                 train_data = None, train_labels = None,
                 test_data = None, test_labels = None,
                  optimizer = None,
                 epochs = 1,train = True, validate = False,
                shuffle = True) :
    
    if train == True :
        loss_history = []
        acc_history = []
        val_loss_history = []
        val_acc_history = []
        
    for epoch in range(epochs) :
        if train == True :
            #=========================TRAINING=================================#
            start_time_epoch = time.time()
            
            model.train()
    
            print("Epoch:", epoch,"/",epochs-1,"===============================================")
        
            running_loss = 0.0
            running_corrects = 0.0
            
            batches = data_gen_1(data=train_data,labels=train_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
        
                inputs,classes,batch_num = data

                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)
                    
                inputs = inputs.float()

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()

                inputs = inputs.view(inputs.size(0), -1)
                    
                # calulate outputs and losses
                outputs = model(inputs)
                loss = criterion(outputs,classes)       

                # autograd
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # statistics
                batch_loss = loss.data.item()
                _,preds = torch.max(outputs.data,1)
                batch_corrects = torch.sum(preds == classes.data)
                
                running_loss += batch_loss
                running_corrects += batch_corrects

                print('Batch {:d}/{:d} - Loss: {:.4f} Acc: {:.4f} - Time : {:.2f}s'.format(i+1,batch_num,
                             batch_loss/len(classes), float(batch_corrects)/len(classes), time.time() - start_time),end='\r')

            epoch_loss = running_loss / len(train_labels)
            epoch_acc = running_corrects.data.item() / len(train_labels)
            #
            
            loss_history.append(epoch_loss)
            acc_history.append(epoch_acc)
            
            print('Epoch {:d} completed in {:.2f} seconds ! Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch , time.time() - start_time_epoch, epoch_loss, epoch_acc))
            
        if validate == True :
            #=========================VALIDATING=================================#
            
            model.eval()
            
            val_loss = 0.0
            val_corrects = 0.0
            
            batches = data_gen_1(data=test_data,labels=test_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
                
                inputs,classes,batch_num = data
                
                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)
                    
                inputs = inputs.float()

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()
                    
                inputs = inputs.view(inputs.size(0), -1)

                outputs = model(inputs)
                loss = criterion(outputs,classes)        
                _,preds = torch.max(outputs.data,1)

                # statistics

                val_loss += loss.data.item()
                val_corrects += torch.sum(preds == classes.data)
                
                print('Validating batch {:d}/{:d} - {:.2f}s ...'.format(i+1,batch_num
                                                                , time.time() - start_time), end="\r")

            val_epoch_loss = val_loss / len(test_labels)
            val_epoch_acc = val_corrects.data.item() / len(test_labels)
            # 

            print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(
                             val_epoch_loss,val_epoch_acc))
            
            if train == False :
                return
            else :
                val_loss_history.append(val_epoch_loss)
                val_acc_history.append(val_epoch_acc)
    
    if train == False :
        return 'On fait rien!'
    elif validate == False :
        return loss_history, acc_history
    else :
        return loss_history, acc_history,val_loss_history,val_acc_history

In [18]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
# optimizer = torch.optim.SGD(net1.parameters(),lr = lr,momentum = 0.9)
optimizer = torch.optim.Adam(net1.parameters())

In [19]:
%%time
history = train_model_1(net1,
                train_data=text_feat_train ,train_labels=valid_labels_train , 
                test_data = text_feat_val , test_labels = valid_labels_val ,
                epochs=5, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=True, validate = True,
                shuffle=True )
print("DONE !")

Epoch: 0 / 9 ===============================================
Epoch 0 completed in 70.42 seconds ! Loss: 0.0309 Acc: 0.5306
Val Loss: 0.0157 Val Acc: 0.7637 ...
Epoch: 1 / 9 ===============================================
Epoch 1 completed in 70.95 seconds ! Loss: 0.0096 Acc: 0.8518
Val Loss: 0.0138 Val Acc: 0.7946 ...
Epoch: 2 / 9 ===============================================
Epoch 2 completed in 71.01 seconds ! Loss: 0.0054 Acc: 0.9198
Val Loss: 0.0138 Val Acc: 0.8025 ...
Epoch: 3 / 9 ===============================================
Epoch 3 completed in 71.06 seconds ! Loss: 0.0042 Acc: 0.9376
Val Loss: 0.0143 Val Acc: 0.8022 ...
Epoch: 4 / 9 ===============================================
Epoch 4 completed in 70.94 seconds ! Loss: 0.0036 Acc: 0.9453
Val Loss: 0.0149 Val Acc: 0.8018 ...
Epoch: 5 / 9 ===============================================
Epoch 5 completed in 70.99 seconds ! Loss: 0.0033 Acc: 0.9496
Val Loss: 0.0149 Val Acc: 0.8057 ...
Epoch: 6 / 9 ===========================

KeyboardInterrupt: 

In [20]:
history = train_model_1(net1,
                train_data=text_feat_train ,train_labels=valid_labels_train , 
                test_data = text_feat_val , test_labels = valid_labels_val ,
                epochs=10, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=False, validate = True,
                shuffle=True )

Val Loss: 0.0161 Val Acc: 0.8069 ...


### Train on Image features

In [48]:
model_vgg = models.vgg16(pretrained=True)

In [49]:
model_vgg.classifier._modules['6'] = nn.Linear(4096, 101)

In [50]:
if use_gpu:
    model_vgg = model_vgg.cuda()

In [51]:
optimizer = torch.optim.SGD(model_vgg.classifier.parameters(),lr = lr,momentum = 0.9)
# optimizer = torch.optim.Adam(model_vgg.classifier.parameters())

In [52]:
print(model_vgg.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=101, bias=True)
)


In [53]:
for param in model_vgg.classifier.parameters():
    param.requires_grad = True

In [54]:
%%time
history = train_model_1(model_vgg.classifier,
                train_data=valid_conv_feat_train  ,train_labels=valid_labels_train , 
                test_data = valid_conv_feat_val  , test_labels = valid_labels_val ,
                epochs=5, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=True, validate = True,
                shuffle=True )

Epoch: 0 / 9 ===============================================
Epoch 0 completed in 61.91 seconds ! Loss: 0.0504 Acc: 0.2560
Val Loss: 0.0420 Val Acc: 0.3759 ...
Epoch: 1 / 9 ===============================================
Epoch 1 completed in 61.94 seconds ! Loss: 0.0438 Acc: 0.3401
Val Loss: 0.0406 Val Acc: 0.4154 ...
Epoch: 2 / 9 ===============================================
Epoch 2 completed in 62.06 seconds ! Loss: 0.0390 Acc: 0.4026
Val Loss: 0.0403 Val Acc: 0.4244 ...
Epoch: 3 / 9 ===============================================
Epoch 3 completed in 62.72 seconds ! Loss: 0.0347 Acc: 0.4588
Val Loss: 0.0400 Val Acc: 0.4315 ...
Epoch: 4 / 9 ===============================================
Epoch 4 completed in 62.13 seconds ! Loss: 0.0310 Acc: 0.5140
Val Loss: 0.0393 Val Acc: 0.4341 ...
Epoch: 5 / 9 ===============================================
Epoch 5 completed in 62.15 seconds ! Loss: 0.0278 Acc: 0.5620
Val Loss: 0.0403 Val Acc: 0.4319 ...
Epoch: 6 / 9 ===========================

KeyboardInterrupt: 

In [55]:
train_model_1(model_vgg.classifier,
                train_data=valid_conv_feat_train  ,train_labels=valid_labels_train , 
                test_data = valid_conv_feat_val  , test_labels = valid_labels_val ,
                epochs=10, batch_size = 32,
                optimizer=optimizer,criterion=criterion ,
                train=False, validate = True,
                shuffle=True )

Val Loss: 0.0404 Val Acc: 0.4382 ...


### Fusion

In [56]:
class myFusionNet(nn.Module):
    def __init__(self):
        super(myFusionNet, self).__init__()
#         self.fc1 = nn.Sequential(
#             nn.Linear(60033, 512),
#             nn.ReLU(True),
#             nn.Dropout(0.3),
#             nn.Linear(512, 128),
#             nn.ReLU(True),
#             nn.Dropout(0.3),
#             nn.Linear(128, 101),
#         ) 
#         self.fc2 = nn.Sequential(
#             nn.Linear(25088, 4096),
#             nn.ReLU(True),
#             nn.Dropout(0.3),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Dropout(0.5),
#             nn.Linear(4096, 101),
#         ) 
        self.ratio = torch.tensor(0.8, requires_grad=True, device="cuda")

    def forward(self, x):
        # x should have shape (?,60033+25088)
        x1 = x[:,:60033]
        x2 = x[:,-25088:]
        #x1 = self.fc1(x1)
        #x2 = self.fc2(x2)
        x1 = net1(x1)
        x2 = model_vgg.classifier(x2)
        out = self.ratio*x1 + (1-self.ratio)*x2
        # out = torch.cat((x1,x2),1)
        # out = self.classifier(out)
        return out

In [57]:
mynet = myFusionNet()
print(mynet)

myFusionNet()


In [58]:
if use_gpu:
    mynet = mynet.cuda()

### Loss & optimizer

In [59]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = torch.optim.SGD(list([mynet.ratio]),lr = lr,momentum = 0.9)

### Define datagen + train_model for data fusion

In [60]:
def data_gen(data1,data2,labels,batch_size=32,shuffle=True):
    """
    data1 expected to be text_feat of type csr_matrix
    data2 expected to be conv_feat of type ndarray
    """
    labels = np.array(labels)
    if shuffle:
        index = np.random.permutation(len(labels))
        data1 = data1[index]
        data2 = data2[index]
        labels = labels[index]
    for idx in range(0,len(labels),batch_size):
        yield(np.concatenate((data1[idx:idx+batch_size].toarray(), data2[idx:idx+batch_size]), axis=1)
              ,labels[idx:idx+batch_size],
             int(len(labels) / batch_size) + (len(labels) % batch_size > 0))

In [61]:
def train_model_2(model, criterion,
                 train_data1 = None,train_data2=None, train_labels = None,
                 test_data1 = None,test_data2=None, test_labels = None,
                  optimizer = None,
                 epochs = 1,train = True, validate = False,
                shuffle = True) :
    
    if train == True :
        loss_history = []
        acc_history = []
        val_loss_history = []
        val_acc_history = []
        
    for epoch in range(epochs) :
        if train == True :
            #=========================TRAINING=================================#
            start_time_epoch = time.time()
            
            model.train()
    
            print("Epoch:", epoch,"/",epochs-1,"===============================================")
        
            running_loss = 0.0
            running_corrects = 0.0
            
            batches = data_gen(data1=train_data1,data2=train_data2,labels=train_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
        
                inputs,classes,batch_num = data

                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)
                    
                inputs = inputs.float()

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()

                inputs = inputs.view(inputs.size(0), -1)
                    
                # calulate outputs and losses
                outputs = model(inputs)
                loss = criterion(outputs,classes)       

                # autograd
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # statistics
                batch_loss = loss.data.item()
                _,preds = torch.max(outputs.data,1)
                batch_corrects = torch.sum(preds == classes.data)
                
                running_loss += batch_loss
                running_corrects += batch_corrects

                print('Batch {:d}/{:d} - Loss: {:.4f} Acc: {:.4f} - Time : {:.2f}s'.format(i+1,batch_num,
                             batch_loss/len(classes), float(batch_corrects)/len(classes), time.time() - start_time),end='\r')

            epoch_loss = running_loss / len(train_labels)
            epoch_acc = running_corrects.data.item() / len(train_labels)
            #
            
            loss_history.append(epoch_loss)
            acc_history.append(epoch_acc)
            
            print('Epoch {:d} completed in {:.2f} seconds ! Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch , time.time() - start_time_epoch, epoch_loss, epoch_acc))
            
        if validate == True :
            #=========================VALIDATING=================================#
            
            model.eval()
            
            val_loss = 0.0
            val_corrects = 0.0
            
            batches = data_gen(data1=test_data1,data2=test_data2,labels=test_labels,shuffle=shuffle)
            
            #batch_num = len(list(batches))

            for i,data in enumerate(batches) :
                start_time = time.time()
                
                inputs,classes,batch_num = data
                
                if  isinstance(inputs, (list, np.ndarray)) :
                    inputs , classes = torch.from_numpy(inputs), torch.from_numpy(classes)
                    
                inputs = inputs.float()

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()
                    
                inputs = inputs.view(inputs.size(0), -1)

                outputs = model(inputs)
                loss = criterion(outputs,classes)        
                _,preds = torch.max(outputs.data,1)

                # statistics

                val_loss += loss.data.item()
                val_corrects += torch.sum(preds == classes.data)
                
                print('Validating batch {:d}/{:d} - {:.2f}s ...'.format(i+1,batch_num
                                                                , time.time() - start_time), end="\r")

            val_epoch_loss = val_loss / len(test_labels)
            val_epoch_acc = val_corrects.data.item() / len(test_labels)
            # 

            print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(
                             val_epoch_loss,val_epoch_acc))
            
            if train == False :
                return
            else :
                val_loss_history.append(val_epoch_loss)
                val_acc_history.append(val_epoch_acc)
    
    if train == False :
        return 'On fait rien!'
    elif validate == False :
        return loss_history, acc_history
    else :
        return loss_history, acc_history,val_loss_history,val_acc_history

### Training

In [62]:
history = train_model_2(model=mynet,criterion=criterion,
              train_data1 = text_feat_train,train_data2=valid_conv_feat_train, train_labels = valid_labels_train,
                 test_data1 = text_feat_val,test_data2= valid_conv_feat_val, test_labels = valid_labels_val,
                  optimizer = optimizer,
                 epochs = 10,train = True, validate = True,
                shuffle = True)

Epoch: 0 / 9 ===============================================
Epoch 0 completed in 111.89 seconds ! Loss: 0.0015 Acc: 0.9941
Val Loss: 0.0175 Val Acc: 0.8642 ...
Epoch: 1 / 9 ===============================================
Epoch 1 completed in 112.06 seconds ! Loss: 0.0015 Acc: 0.9942
Val Loss: 0.0178 Val Acc: 0.8626 ...
Epoch: 2 / 9 ===============================================


KeyboardInterrupt: 

In [63]:
mynet.ratio

tensor(0.5083, device='cuda:0', requires_grad=True)

In [64]:
history = train_model_2(model=mynet,criterion=criterion,
              train_data1 = text_feat_train,train_data2=valid_conv_feat_train, train_labels = valid_labels_train,
                 test_data1 = text_feat_val,test_data2= valid_conv_feat_val, test_labels = valid_labels_val,
                  optimizer = optimizer,
                 epochs = 10,train = False, validate = True,
                shuffle = True)

Val Loss: 0.0176 Val Acc: 0.8637 ...
